In [1]:
"""Energy price pred package returns selling price of electricity (back to grid)

To install, copy the package folder: energy_price_pred

pip install . (from the shell to install package)

###### then run the following commands #########

from energy_price_pred.energypricepred import *

download_file(file, save_path)

df_price, train, test = process_df(file, previous_days=30*18)   ### example uses last 18 months and returns electricity selling price dataframe with splits

model, forecast_y_df, df_cv, df_p = ml_model(df_price, forecast_days=14, year_seasonality_mode=4) ### returns trained model

pred_y_df,date1, date2, forecast_days = pred(df_price, model, forecast_start_date='2024-03-19', forceast_end_date='2024-03-31') ### returns forecasted electricity selling price pred_y_df

### Hourly selling price of electricity is obtained as follows,

hourly_selling_data = pred_y_df.resample('H', on='ds').mean()

hourly_buying_data = hourly_selling_data.copy()

hourly_buying_data = hourly_selling_data.copy()

""";

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from energy_price_pred.energypricepred import *

/home/rahuliyer/.pyenv/versions/3.11.6/envs/market/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [ ]:
download_file(file, save_path)

In [4]:
df_price, train, test = process_df(file, previous_days=30*12)

2019-05-15 23:00:00+00:00
17280
df created including time history of electricity export prices between 2023-03-29 23:00:00 and 2024-03-23 22:30:00 i.e. for the last 359 days 23:30:00 days


In [5]:
model, forecast_y_df, df_cv, df_p = ml_model(df_price, year_seasonality_mode=4)

12:22:42 - cmdstanpy - INFO - Chain [1] start processing
12:22:45 - cmdstanpy - INFO - Chain [1] done processing
Seasonality has period of 365.25 days which is larger than initial window. Consider increasing initial.


performing cross-validation using, initial: 180, horizon:36, and period:18


  0%|          | 0/8 [00:00<?, ?it/s]12:22:48 - cmdstanpy - INFO - Chain [1] start processing
12:22:52 - cmdstanpy - INFO - Chain [1] done processing
 12%|█▎        | 1/8 [00:05<00:36,  5.26s/it]12:22:53 - cmdstanpy - INFO - Chain [1] start processing
12:22:56 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 2/8 [00:08<00:25,  4.24s/it]12:22:56 - cmdstanpy - INFO - Chain [1] start processing
12:22:58 - cmdstanpy - INFO - Chain [1] done processing
 38%|███▊      | 3/8 [00:11<00:17,  3.42s/it]12:22:59 - cmdstanpy - INFO - Chain [1] start processing
12:23:00 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 4/8 [00:13<00:11,  2.93s/it]12:23:01 - cmdstanpy - INFO - Chain [1] start processing
12:23:04 - cmdstanpy - INFO - Chain [1] done processing
 62%|██████▎   | 5/8 [00:16<00:08,  2.97s/it]12:23:04 - cmdstanpy - INFO - Chain [1] start processing
12:23:06 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 6/8 [00:18<00:05,  2.74s/it]12:23:07 - cmds

In [6]:
type(df_price['ds'][df_price.index[0]])

pandas._libs.tslibs.timestamps.Timestamp

In [7]:
df_price.tail()

,ds,y
85146,2024-03-23 20:30:00,4.14
85147,2024-03-23 21:00:00,5.50
85148,2024-03-23 21:30:00,3.37
85149,2024-03-23 22:00:00,5.00
85150,2024-03-23 22:30:00,2.16


In [8]:
# date1 = datetime.strptime('2024-03-23 20:30:00', "%Y-%m-%d %H:%M:%S")
# date2 = datetime.strptime('2024-03-23 22:30:00', "%Y-%m-%d %H:%M:%S")
# forecast_y_df[(forecast_y_df['ds']>= date1)  & (forecast_y_df['ds']<=date2)]

In [10]:
pred_y_df_sell, hourly_data_sell, hourly_data_buy, date1, date2, forecast_days = pred(model, forecast_start_date='2024-03-23', forecast_days=7, freq='H')

7


In [11]:
date1, date2, forecast_days

(datetime.date(2024, 3, 30), datetime.date(2024, 3, 23), 7)

In [12]:
pred_y_df_sell.tail()

,ds,yhat,yhat_lower,yhat_upper
17443,2024-03-30 18:30:00,10.224624,6.375620,14.514448
17444,2024-03-30 19:30:00,8.135847,3.765570,12.461687
17445,2024-03-30 20:30:00,6.626595,2.090818,10.763237
17446,2024-03-30 21:30:00,6.145889,2.016349,10.494701
17447,2024-03-30 22:30:00,6.266288,2.188833,10.563295


In [13]:
date1, date2, forecast_days

(datetime.date(2024, 3, 30), datetime.date(2024, 3, 23), 7)

In [ ]:
ax = model.plot(pred_y_df_sell)

In [ ]:
import matplotlib.pyplot as plt

days =14

date = df_price['ds'][df_price.index[-1]]- timedelta(days=days)
date_end = date + timedelta(days=days+days)
# date
df_price_date=df_price.loc[df_price['ds'] >= date ].copy()
forecast_date=pred_y_df_sell.loc[(pred_y_df_sell['ds'] >= date ) & (pred_y_df_sell['ds'] <= date_end)].copy()

plt.figure(figsize=(15,8))
plt.plot(df_price_date['ds'], df_price_date['y'], label='df_price')
plt.plot(forecast_date['ds'], forecast_date['yhat'], label='predictions', color = 'red')
plt.legend(loc='best')

In [ ]:
df_p.tail()

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='smape')

In [ ]:
df_price, train, test = process_df(file, previous_days=30*18)

In [ ]:
model, forecast_y_df, df_cv, df_p = ml_model(df_price, forecast_days=14, year_seasonality_mode=4, freq='H')

In [ ]:
pred_y_df_sell, hourly_data_sell, hourly_data_buy, date1, date2, forecast_days = pred(df_price, model, forecast_start_date='2024-03-23', forceast_end_date='2024-04-06')

In [ ]:
ax = model.plot(pred_y_df_sell)

In [ ]:
df_price

In [ ]:
pred_y_df_sell

In [ ]:
import matplotlib.pyplot as plt

days =14

date = df_price['ds'][df_price.index[-1]]- timedelta(days=days)
date_end = date + timedelta(days=days+days)
# date
df_price_date=df_price.loc[df_price['ds'] >= date ].copy()
forecast_date=pred_y_df_sell.loc[(pred_y_df_sell['ds'] >= date ) & (pred_y_df_sell['ds'] <= date_end)].copy()
# test_date=test.loc[test['ds'] <= date_end ].copy()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(df_price_date['ds'], df_price_date['y'], label='df_price')
plt.plot(forecast_date['ds'], forecast_date['yhat'], label='predictions', color = 'red')
plt.legend(loc='best')

In [ ]:
fig = plot_cross_validation_metric(df_cv, metric='smape')

In [ ]:
"""Energy price pred package returns selling price of electricity (back to grid)

To install, copy the package folder: energy_price_pred

pip install . (from the shell to install package)

###### then run the following commands #########

from energy_price_pred.energypricepred import *

download_file(file, save_path)

df_price, train, test = process_df(file, previous_days=30*18)   ### example uses last 18 months and returns electricity selling price dataframe with splits

model, forecast_y_df, df_cv, df_p = ml_model(df_price, forecast_days=14, year_seasonality_mode=4) ### returns trained model

pred_y_df,date1, date2, forecast_days = pred(df_price, model, forecast_start_date='2024-03-19', forceast_end_date='2024-03-31') ### returns forecasted electricity selling price pred_y_df

### Hourly selling price of electricity is obtained as follows,

hourly_selling_data = pred_y_df.resample('H', on='ds').mean()

hourly_buying_data = 2*hourly_selling_data

"""


In [ ]:
hourly_data = pred_y_df.resample('H', on='ds').mean()

In [ ]:
hourly_data

In [ ]:
hourly_data_buy= hourly_data*2
# hourly_data_buy['y_hat']=hourly_data_buy['y']*2
hourly_data_buy
